In [ ]:
import requests
import pandas as pd 
import folium
from folium import FeatureGroup, features, CircleMarker
import numpy as np
from matplotlib import colors as mcolors
from haversine import haversine
from folium import IFrame
from IPython.display import Markdown, display
from math import radians, sin, cos, asin, atan2, degrees


In [ ]:
def calculate_bearing(point1, point2):
        lat1 = point1[0]
        lon1 = point1[1]

        lat2 = point2[0]
        lon2 = point2[1]
        lat1, lon1, lat2, lon2 = map(radians, (lat1, lon1, lat2, lon2))
        y = sin(lon2-lon1)*cos(lat2);
        x = cos(lat1)*sin(lat2) - sin(lat1)*cos(lat2)*cos(lon2-lon1);
        bearing = degrees(atan2(y, x))
        return bearing

In [ ]:
def calculate_coordinates(departure_point, bearing, distance):
        R = earth_radius = 6371
        d = distance
        lat1 = radians(departure_point[0]) #Current lat point converted to radians
        lon1 = radians(departure_point[1]) #Current long point converted to radians
        bearing = radians(bearing)
        lat2 = asin( sin(lat1)*cos(d/R) + cos(lat1)*sin(d/R)*cos(bearing))
        lon2 = lon1 + atan2(sin(bearing)*sin(d/R)*cos(lat1), cos(d/R)-sin(lat1)*sin(lat2))
        lat2 = degrees(lat2)
        lon2 = degrees(lon2)
        return (round(lat2, 4), round(lon2, 4)) 

In [ ]:
p1,p2 = tunnelnodes[8][0],tunnelnodes[8][-1]
b = calculate_bearing(p1,p2)
b

In [ ]:
p1

In [ ]:
calculate_coordinates(p1, 90, 2)

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
CircleMarker(p1,color='red').add_to(m)
CircleMarker(p2,color='red').add_to(m)
folium.PolyLine([p2,calculate_coordinates(p2,b+90, 0.5),calculate_coordinates(p1,b+90, 0.5),
                 p1,calculate_coordinates(p1,b-90, 0.5),
                 calculate_coordinates(p2,b-90, 0.5),p2],color='green',radius=2).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
query=\
"""
[out:json];
way
  [highway!~"(pedestrian|path|footway|steps|via_ferrata|service|construction|unclassified)"]
  [tunnel][bicycle=no]
  (46.37346430137335,11.211547851562498,47.184112659842015,12.60955810546875);
(._;>;);
out;"""

In [ ]:
query=\
"""
[out:json];
way
  [highway!~"(pedestrian|path|footway|steps|via_ferrata|service|construction|unclassified)"]
  [tunnel][ref=SS49] (46.37346430137335,11.211547851562498,47.184112659842015,12.60955810546875);
(._;>;);
out;"""

In [ ]:
r = requests.get('http://overpass-api.de/api/interpreter?data='+query)
print(r.status_code)
ores = r.json()

In [ ]:
tunnels =[e for e in ores['elements'] if e['type']=='way']
nodes = [e for e in ores['elements'] if e['type']=='node']
len(tunnels)

In [ ]:
tunnelnodes = []
for tunnel in tunnels:
    nt = []
    for nodeid in tunnel['nodes']:
        for node in nodes:
            if node['id'] == nodeid:
                nt.append([node['lat'],node['lon']])
    tunnelnodes.append(nt)

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
for i,tunnel in enumerate(tunnels):
    folium.PolyLine(tunnelnodes[i],color='red',popup=str(i)).add_to(m)
folium.Marker([46.732016, 12.198863]).add_to(m)    
m.fit_bounds(m.get_bounds())
m

In [ ]:
tunnels[8]

## All forbidden Tunnels

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
for i,tunnel in enumerate(tunnels):
    folium.PolyLine(tunnelnodes[i],color='red').add_to(m)
folium.Marker([46.732016, 12.198863]).add_to(m)    
m.fit_bounds(m.get_bounds())
m

In [ ]:
mypath='./2019-08-05/Tracks/'

In [ ]:
racers = pd.read_pickle('racers.pkl')

In [ ]:
route={}
for racer in racers['Name']:
    try:
        ll =[l[0] for i,l in enumerate(pd.read_pickle(mypath+racer+'.pkl').values.tolist())]
        route[racer] = ll
    except:
        print (racer)

In [ ]:
mycolors = list(mcolors.cnames.keys())

In [ ]:
tunnellength = [sum(list(map(lambda x,y:haversine(x,y),tunnel[1:],tunnel[:-2]))) for tunnel in tunnelnodes]

In [ ]:
bg =[]
for k,racer in racers.iterrows():
    try:
        if racer['Bib']=='CC1' or racer['Bib']=='CC2':
            continue
        for i,l in enumerate(route[racer['Name']]):
            for j,tunnel in enumerate(tunnelnodes):
                d = haversine(l,tunnel[0],'km')
                if d < 2:
                    bg.append([j,d,l,i,racer['Name'],racer['Bib']])
                d = haversine(l,tunnel[-1],'km')                    
                if d < 2:
                    bg.append([j,d,l,i,racer['Name'],racer['Bib']])
    except:
        print(racer['Name'])

## Forbidden tunnels where riders came close

In [ ]:
bgt =list(set([e[0] for e in bg]))
bgt

In [ ]:
def popup(t,show=False):
    return folium.Popup('%s' % (t),show=show)

In [ ]:
[[i,tunnels[i]['tags']['ref']] for i in bgt if 'ref' in tunnels[i]['tags']]

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
for i in bgt:
    name = tunnels[i]['tags']['name'] if 'name' in tunnels[i]['tags'] else ''
    folium.PolyLine(tunnelnodes[i],color='red',popup=popup(name)).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[0]
display(Markdown('# '+tunnels[ibgt]['tags']['name']+' '+tunnels[ibgt]['tags']['ref']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
m.fit_bounds(m.get_bounds())
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m

Josh Murphy, Yiqun_Fu and Purim_Sirimachan look suspicious

In [ ]:
name = 'Josh_Murphy'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name][pt[0]-1:pt[-1]+1]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[1],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[1][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[1][-1],radius=2,color='purple',fillcolor='purple').add_to(m)

[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
name = 'Yiqun_Fu'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name][pt[0]-1:pt[-4]]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[1],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[1][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[1][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
name = 'Purim_Sirimachan'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name][pt[0]-1:pt[-1]]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[1],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[1][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[1][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[1]
display(Markdown('# '+tunnels[ibgt]['tags']['name']+' '+tunnels[ibgt]['tags']['ref'] ))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
name = 'Rados__aw_Rog____'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name][pt[0]:pt[-1]]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
name = 'Tom_Probert'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name][pt[0]:pt[-1]]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[2]
display(Markdown('# '+tunnels[ibgt]['tags']['name']+' '+tunnels[ibgt]['tags']['ref'] ))


In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color='red',radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
name = 'Jesko_von_Werthern'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
m.fit_bounds(m.get_bounds())
#[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m

In [ ]:
ibgt = bgt[3]
display(Markdown('# '+tunnels[ibgt]['tags']['name']+' '+tunnels[ibgt]['tags']['ref']))
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[4]
display(Markdown('# '+tunnels[ibgt]['tags']['name']+' '+tunnels[ibgt]['tags']['ref']))
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[5]
display(Markdown('# '+tunnels[ibgt]['tags']['name']+' '+tunnels[ibgt]['tags']['ref']))
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
name = 'Yiqun_Fu'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
m.fit_bounds(m.get_bounds())
#[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m

In [ ]:
ibgt = bgt[6]
display(Markdown('# '+tunnels[ibgt]['tags']['name']+' '+tunnels[ibgt]['tags']['ref']))
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[7]
display(Markdown('# '+tunnels[ibgt]['tags']['name']+' '+tunnels[ibgt]['tags']['ref']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
tunnels[ibgt]['tags']

In [ ]:
ibgt = bgt[8]
display(Markdown('# '+tunnels[ibgt]['tags']['ref']))

There seems to be a path around the tunnel

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
name = 'Fausto_Ottoboni'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name][pt[0]-1:pt[-1]]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
m.fit_bounds(m.get_bounds())
[CircleMarker(l,popup=popup(name),radius=2).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m

In [ ]:
name = 'Massimo_Varini'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name][pt[0]:pt[-1]]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
m.fit_bounds(m.get_bounds())
[CircleMarker(l,popup=popup(name),radius=2).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m

In [ ]:
len(bgt)